In [ ]:
!pip install openai-agents sendgrid

In [ ]:
from agents import Agent, Runner, trace, function_tool, ImageGenerationTool, RunContextWrapper
import os
import asyncio
from typing import TypedDict, Dict
from pydantic import Field, BaseModel
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail, Email, To, Content, Attachment, FileContent, FileName, FileType, Disposition, ContentId

from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['SENDGRID_API_KEY'] = userdata.get('SENDGRID_API_KEY')

from agents import set_tracing_export_api_key

set_tracing_export_api_key(os.getenv("OPENAI_API_KEY"))



In [ ]:
# Improved instructions thanks to student Guillermo F.
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

description = "Write a cold sales email"

sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the best cold sales email of the generated mails using the sales_agent tools.

Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready. Do not execute them more than once.

2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.

3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.

Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must output exactly ONE email — never more than one.
"""
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3]

sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    #handoffs=handoffs,
    model="gpt-4o-mini")

image_generation_tool = ImageGenerationTool(
    tool_config={
        'type': 'image_generation',
        'quality': 'low',  # Options: 'low', 'medium', 'high', 'auto'
        'size': '1024x1024',  # Or '1024x1536', '1536x1024', 'auto'
        'output_format': 'png',  # Or 'webp', 'jpeg'
    }
)

image_agent = Agent(name='Image_Generator',
                        instructions="You will create a small image that goes well with the email attached using your image_generation_tool.",                        model="gpt-4o",
                        tools=[image_generation_tool],
                        )

html_instructions = """You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design.\
Your output should strictly be the body in HTML format (without ```html nor ```), including the tag <img src="cid:my_image" alt="My Image" />"""
html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")

class Ctx(BaseModel):
    html_body: str
    img_64: str

    def __getitem__(self, key: str):
        return self.model_dump()[key]

@function_tool
def send_html_tool(ctx: RunContextWrapper[Ctx])-> Dict[str, str]:
    """
    Send an HTML email with an embedded image.

    Args:
        ctx: Dictionary containing 'html_body' (HTML content) and 'img_64' (base64 encoded image)

    Returns:
        Dictionary with 'status' indicating success
    """
    context_dict = ctx.context

    html_body = context_dict["html_body"]
    encoded_image = context_dict["img_64"]

    sg = SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))

    message = Mail(
        from_email=Email("javier.otero.marquez@gmail.com"),
        to_emails=To("javiomotero@gmail.com"),
        subject="Your Message",
        html_content=html_body
    )
    attachment = Attachment(
        FileContent(encoded_image),
        FileName('image.jpg'),
        FileType('image/jpeg'),
        Disposition('inline'),
        ContentId('my_image')
    )

    message.add_attachment(attachment)

    response = sg.send(message)
    return {"status": "success"}

mailer_instructions ="You are an email sender. Use send_html_tool to send the prepared email. All required data is already available in context."
emailer_agent = Agent(
    name="Email Manager",
    instructions=mailer_instructions,
    tools=[send_html_tool],
    model="gpt-4o-mini",
    )
message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    best_mail_result = await Runner.run(sales_manager, message)
    best_mail_str = best_mail_result.final_output
    image_result = await Runner.run(image_agent, best_mail_str)
    img_64_str = image_result.raw_responses[0].output[0].result

    html_body = await Runner.run(html_converter, best_mail_str)
    html_body_str = html_body.final_output
    emailer = await Runner.run(
        starting_agent=emailer_agent,
        input="Use your send_html_tool to send the prepared email with the image.",
        context=Ctx(html_body=html_body_str, img_64=img_64_str)
    )
